# CS6405 - Data Mining - First Assignment

Lecturer: Dr Andrea Visentin

TA: Andrea Balogh

## Submission

This assignment is **due on 28/02/23 at 23:59**. You should submit a single .ipnyb file with your python code and analysis electronically via Canvas.
Please note that this assignment will account for **25 Marks** of your module grade.

## Declaration

By submitting this assignment. I agree to the following:

<font color="red">“I have read and understand the UCC academic policy on plagiarism and I agree to the
requirements set out thereby in relation to plagiarism and referencing. I confirm that I
have referenced and acknowledged properly all sources used in preparation of this
assignment.
I declare that this assignment is entirely my own work based on my personal study. I
further declare that I have not engaged the services of another to either assist me in, or
complete this assignment”</font>

## Specification 

The objective of this project is to build a k-Nearest Neighbour algorithm that takes as input training and test dataset and will predict the target variable with a reasonable degree of accuracy.


You will find a template for your python code in Canvas along with the dataset. In this project, you must implement your own machine learning library, and therefore **you are only allowed to use the following python packages and libraries: NumPy and pandas.**

This project focuses on a modified version of the Real estate valuation data set (https://archive.ics.uci.edu/ml/datasets/Real+estate+valuation+data+set). 

You can find the correct csv that you should use for the assignment here:
https://github.com/andvise/DataAnalyticsDatasets/blob/main/real_estate.csv

# Tasks:



# Data Preparation (5 marks)



1.  Load the dataset on Colab 


In [1]:
# Import of neccesary libraries
import pandas as pd
import numpy as np

estate_df = pd.read_csv('https://github.com/andvise/DataAnalyticsDatasets/blob/16a2159cf151c19f7a3878cea06ed27b5cb4fd9d/real_estate.csv?raw=true')




---





2. Display the attributes' name and their data type




In [2]:
print(estate_df.dtypes)

num                            int64
transaction_date             float64
house_age                    float64
distance_MRT_station         float64
number_convenience_stores      int64
latitude                     float64
longitude                    float64
house_price_per_unit         float64
dtype: object


On the left side are listed the names of the attributes VS on the right are their respective data types.



---



3. Delete the columns **num** and **transaction_date**



In [3]:
estate_df = estate_df[['house_age', 'distance_MRT_station',
       'number_convenience_stores', 'latitude', 'longitude',
       'house_price_per_unit']]

# need double square brackets to specify multiple column names within a list

'Removing' columns by modifying the estate_df to contain all the columns except 'num' and 'transaction_date'.




---



4. Use **house_price_per_unit** as the target value and the rest of the data frame as
features


Target value means label.
Rest of df means remaining attributes i.e. columns.

In [4]:
labels = estate_df['house_price_per_unit']

data = estate_df[['house_age', 'distance_MRT_station', 'number_convenience_stores',
       'latitude', 'longitude']]



---



5. Divide your dataset into 80% for training and 20% for test
 

In [5]:
# check shape of dataset
estate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   house_age                  414 non-null    float64
 1   distance_MRT_station       414 non-null    float64
 2   number_convenience_stores  414 non-null    int64  
 3   latitude                   414 non-null    float64
 4   longitude                  414 non-null    float64
 5   house_price_per_unit       414 non-null    float64
dtypes: float64(5), int64(1)
memory usage: 19.5 KB


Noticed 414 entries and 6 columns (5 attributes and 1 label).

In [6]:
# 414 entires; divide into 80% training set VS. 20% test set.

train_data = estate_df[['house_age', 'distance_MRT_station', 'number_convenience_stores',
       'latitude', 'longitude']][:331] #take rows 0:331, as ~80% of 414 is 331.2

train_labels = estate_df['house_price_per_unit'][:331] 

test_data = estate_df[['house_age', 'distance_MRT_station', 'number_convenience_stores',
       'latitude', 'longitude']][331:] # take rows 331: till the end (414th row) as this is remaining ~20% of data for the attributes

test_labels = estate_df['house_price_per_unit'][331:] 

In [7]:
#to check if division of data into 80-20 worked:
print(train_data.shape)
print(train_labels.shape)
print('--------------------------------')
print(test_data.shape)
print(test_labels.shape)

(331, 5)
(331,)
--------------------------------
(83, 5)
(83,)


331+83 = 414 entries (rows)
Test and train data have 5 attributes (columns) each, whereas their labels have one i.e. 'house_price_per_unit'



---



6. Scale the columns using min-max scalers


In [8]:
# Display the first 5 entries of training data and their labels to observe the range of their values for each attribute

print(train_data.head())  
print('---------------------------------------------------------------------------------')
print(train_labels.head()) 

   house_age  distance_MRT_station  number_convenience_stores  latitude  \
0       32.0              84.87882                         10  24.98298   
1       19.5             306.59470                          9  24.98034   
2       13.3             561.98450                          5  24.98746   
3       13.3             561.98450                          5  24.98746   
4        5.0             390.56840                          5  24.97937   

   longitude  
0  121.54024  
1  121.53951  
2  121.54391  
3  121.54391  
4  121.54245  
---------------------------------------------------------------------------------
0    37.9
1    42.2
2    47.3
3    54.8
4    43.1
Name: house_price_per_unit, dtype: float64


Each attribute has a range of values [min, max]. 

Different attributes having different ranges implies that the distance between neighbours could be dominated by an attribute with relatively large values. 

Therefore creating a function to scale them within a [0,1] range will be useful as to count the attributes equally.  

In [9]:
# Function to perform normalization of values in a df:

def scaling_df(df_subset):
  min_max_scaler = (df_subset - df_subset.min()) / (df_subset.max() - df_subset.min())
  return min_max_scaler

# Pandas automatically applies colomn-wise function in the code above.

The potential issues that arise from processing data this way is that all attributes will have the exact same range. 

This can be problematic because adjusting the data to fit within the same range coudl lead to a loss of information which could in itself have a  meaningful interpretation. 

Also, considering how this is min-max scaling, if there are outliers in the data they can have a disproportionate effect on the scaling.

In [10]:
train_data = scaling_df(train_data)

In [11]:
#to verify whether scaling worked

print(train_data.head)

<bound method NDFrame.head of      house_age  distance_MRT_station  number_convenience_stores  latitude  \
0     0.730594              0.009650                        1.0  0.612907   
1     0.445205              0.044440                        0.9  0.580578   
2     0.303653              0.084514                        0.5  0.667769   
3     0.303653              0.084514                        0.5  0.667769   
4     0.114155              0.057617                        0.5  0.568699   
..         ...                   ...                        ...       ...   
326   0.093607              0.005193                        0.7  0.300147   
327   0.079909              0.115168                        0.3  0.519838   
328   0.363014              0.231344                        0.3  0.454323   
329   0.310502              0.654956                        0.0  0.072496   
330   0.730594              0.177846                        0.0  0.201078   

     longitude  
0     0.714301  
1     0.706

In [12]:
train_labels = scaling_df(train_labels)

In [13]:
test_data = scaling_df(test_data)

In [14]:
test_labels = scaling_df(test_labels)



---



7. Print the shape of the train and test set  

In [15]:
print(train_data.shape)
print(train_labels.shape)
print('-------------------')

print(test_data.shape)
print(test_labels.shape)

(331, 5)
(331,)
-------------------
(83, 5)
(83,)


Receive 331 entries for training set of the 5 attributes, as well as corresponding 331 entires for the training labels (i.e house_price_per_unit).

On the other hand, receive 83 entries (20% of the original estate_df) for test set of the 5 attributes and 83 entries for the test labels.



---



# NN implementation (10 Marks)

1. Implement your NN method. To predict **each** point (query point) of the test set, you need to:


> a. Find the training point with the smallest **Manhattan** distance  with the query point


  
> b. Use as a prediction the target value of the point with the smallest distance to the query point

In [16]:
def NN_implementation(train_data, train_labels, test_data): 

  min_manhattan_all = []
  test_samples =[]
  pred_label = []

  for idx, i in test_data.iterrows(): # idx is the index of the current row, i signifies the contents of that row

    man_dist_lst = [] # empty list stores computed manhattan distance of every training sample (all j's) for given i (test sample)

    for jdx, j in train_data.iterrows():

      man_dist = abs(j['house_age'] - i['house_age']) + abs(j['distance_MRT_station'] - i['distance_MRT_station']) + abs(j['number_convenience_stores'] - i['number_convenience_stores']) + abs(j['latitude'] - i['latitude']) + abs(j['longitude'] - i['longitude'])

      man_dist_lst.append(man_dist) # append computed man_dist to man_dist_lst

    min_manhattan_ith = min(man_dist_lst) # selects only smallest value of manhattan distance for given i

    index_min_k = man_dist_lst.index(min_manhattan_ith) # finding the index of j which has the smallest manhattan distance 

    pred_label.append(train_labels[index_min_k]) # add predicted value of ith element i.e. label of the j with smallest manhattan distance to given i 

    min_manhattan_all.append(min_manhattan_ith) # store smallest manhattan distance of given i (test sample) to a j (training sample), but for all i's

    test_samples.append(idx) # record test sample index value into a list test_samples in order to be able to input it as a column within the manhattan_df


  # creating a data frame to store manhattan distance of test sample, and the training sample closest to it:
  manhattan_df = pd.DataFrame( 
      {'Test_Sample': test_samples, # store as a list all the test sample indexes; this list will represent values for column 'Test Samples'
      'Man_Distance': min_manhattan_all,
      'Pred_Label(Price)' : pred_label 
      }) 

  print(manhattan_df)

  return pred_label



---



2. Compute the mean absolute error (MAE) and the mean squared error (MSE) between the test labels and the predicted
values

In [17]:
# Storing pred_label (the returned value) in a new variable because it is a local variable within the NN_implementation function
# It cannot be accessed directly from outside the function, so its type needs to be checked this way

pred_label = NN_implementation(train_data, train_labels, test_data)

print('--------------------------------------------------------------')

print(type(pred_label))
print(type(test_labels))

    Test_Sample  Man_Distance  Pred_Label(Price)
0           331      0.092242           0.071884
1           332      0.243704           0.390355
2           333      0.090241           0.450409
3           334      0.502746           0.241128
4           335      0.093020           0.317561
..          ...           ...                ...
78          409      0.091749           0.112830
79          410      0.203975           0.410373
80          411      0.232674           0.432211
81          412      0.097266           0.450409
82          413      0.201959           0.467698

[83 rows x 3 columns]
--------------------------------------------------------------
<class 'list'>
<class 'pandas.core.series.Series'>


Seeing that test_labels is a series, pred_label has been converted to series as well. 


In [18]:
pred_labels_ser = pd.Series(pred_label)

print(type(pred_labels_ser))
print('-------------------------------')
print(pred_labels_ser.shape)
print(test_labels.shape)

<class 'pandas.core.series.Series'>
-------------------------------
(83,)
(83,)


Above it has been established that the two series are equivalent in format/size now.



---



In [19]:
#Function to compute Mean Absolute Error

def MAE(pred_label, test_labels):

  pred_labels_ser = pd.Series(pred_label) # converting the predicted labels list to a series in order to be compatible with the test labels

  diff = []

  for i, j in zip(test_labels, pred_labels_ser): # zip aggregates the 2 series into tuples; the for loop iterates over each tuple

      minus = abs(i - j)
      diff.append(minus)

  summed = sum(diff)

  MAE = summed / (test_labels.shape[0]) # test_labels.shape[0] gives len of rows; since test and predicted labels have equal lenghts, using either works.

  print(MAE)

In [20]:
#Function to compute Mean Squared Error

def MSE(pred_label, test_labels):

  pred_labels_ser = pd.Series(pred_label)
  
  diff = []

  for i, j in zip(test_labels, pred_labels_ser): 

      minus = (i - j) ** 2
      diff.append(minus)

  summed = sum(diff)

  MSE = summed / (test_labels.shape[0])

  print(MSE)



---



MAE and MSE behave differently depending on how big the error is.

If the error made is big, MSE penalizes more.

If the error made is small, MSE penalizes less.

MAE is fair on all mistakes.  It is useful when both large and small errors are equally important.

Seeing the relatively small MSE value, it can be assumed that the error produced by processing the real VS predicted label values is farily small.

# k-NN implementation (10 Marks)

1. Extend the previous implementation by using the average of the **k** closest neighbours as a prediction
2. Implement the **Euclidian** distance as well


In [21]:
# Note: All comments that are common in the NN_implementation function have been deleted as to not be redundant

def kNN_implementation(train_data, train_labels, test_data, k):

  import math # to be able to use square root function 

  test_samples =[]
  pred_label = []

  for idx, i in test_data.iterrows():

    eucl_dist_lst = [] 

    for jdx, j in train_data.iterrows():

      # formula for the euclidian distance metric:
      eucl_dist = math.sqrt(((j['house_age'] - i['house_age'])**2) + ((j['distance_MRT_station'] - i['distance_MRT_station']) ** 2) + ((j['number_convenience_stores'] - i['number_convenience_stores'])** 2) + ((j['latitude'] - i['latitude']) ** 2) + ((j['longitude'] - i['longitude'])**2))

      eucl_dist_lst.append(eucl_dist) 


    KNN_min_eucl = [] # empty list to store j number of smallest distance from given i
    index_min_j = [] # empty list to store indexes of all kNNs from given i

    for elem in range(k): # to compute K closest neighbours of ith test sample 

      min_eucl = min(eucl_dist_lst)
      KNN_min_eucl.append(min_eucl) # append smallest euclidan distance of ith test sample to K number of training samples

      index_min = eucl_dist_lst.index(min_eucl)
      index_min_j.append(index_min) # add indexes of K number of training samples with smallest distance to ith sample

      pred_label.append(train_labels[index_min]) # add K number of labels which correspond to K training samples which closest to particular ith sample

      eucl_dist_lst.remove(min_eucl) # delete the minimum value from eucl_dist_lst as to be able to proceed to next min in the following loop

  
    test_samples.append(idx) # to show all KNNs for particular test sample, the sample index had to be multiplied by the K value chosen beforehand


  pred_label_avg = []

  for i in range(0, len(pred_label), k): # loop with a step of k because every k number of elements belongs to the same test sample
      avg = sum(pred_label[i:i+k])/k # finding average of labels of k training samples by considering the sum from KNNs and diving by number of KNNs 
      pred_label_avg.append(avg) 

  #removed the Eucl_Distance column as it seemed a bit redunant since it would store k number of shortest distances from given test sample
  euclidian_df = pd.DataFrame( 
      {'Test_Sample': test_samples, 
      'Pred_Label(Price)' : pred_label_avg 
      }) 

  print(euclidian_df)

  return pred_label_avg



---



3. Test your approach for **k** values = [1, 3, 5, 7] and the two distance measures

> a. Which is the best combination of **k** and distance?



---



### Self-standing function for k-NN with the choice of Euclidean or Manhattan.

In [22]:
def kNN_algorithm(train_data, train_labels, test_data, k, distance_metric):

  import math 

  test_samples =[]
  pred_label = []

  for idx, i in test_data.iterrows():

    dist_lst = [] 

    for jdx, j in train_data.iterrows():

      if distance_metric == 'euclidean':

        dist = math.sqrt(((j['house_age'] - i['house_age'])**2) + ((j['distance_MRT_station'] - i['distance_MRT_station']) ** 2) + ((j['number_convenience_stores'] - i['number_convenience_stores'])** 2) + ((j['latitude'] - i['latitude']) ** 2) + ((j['longitude'] - i['longitude'])**2))
        dist_lst.append(dist)

      elif distance_metric == 'manhattan':
        dist = abs(j['house_age'] - i['house_age']) + abs(j['distance_MRT_station'] - i['distance_MRT_station']) + abs(j['number_convenience_stores'] - i['number_convenience_stores']) + abs(j['latitude'] - i['latitude']) + abs(j['longitude'] - i['longitude'])
        dist_lst.append(dist)
        
      else:
        print('invalid distance metric')


    KNN_min_dist = [] 
    index_min_j = [] 

    for elem in range(k): 

      min_dist = min(dist_lst)
      KNN_min_dist.append(min_dist) 

      index_min = dist_lst.index(min_dist)
      index_min_j.append(index_min) 

      pred_label.append(train_labels[index_min]) 

      dist_lst.remove(min_dist) 

  
    test_samples.append(idx) 


  pred_label_avg = []

  for i in range(0, len(pred_label), k): 
      avg = sum(pred_label[i:i+k])/k  
      pred_label_avg.append(avg) 

  
  kNN_df = pd.DataFrame( 
      {'Test_Sample': test_samples, 
      'Pred_Label(Price)' : pred_label_avg 
      }) 

  print(kNN_df)

  return pred_label_avg



---



# Computation of k = 1,3,5,7 with the Manhattan distance matric

In [23]:
MAE(kNN_algorithm(train_data, train_labels, test_data, k = 1, distance_metric = 'manhattan'), test_labels) 

print('-------------------------------------------------------------------------')

MSE(kNN_algorithm(train_data, train_labels, test_data, k = 1, distance_metric = 'manhattan'), test_labels)

    Test_Sample  Pred_Label(Price)
0           331           0.071884
1           332           0.390355
2           333           0.450409
3           334           0.241128
4           335           0.317561
..          ...                ...
78          409           0.112830
79          410           0.410373
80          411           0.432211
81          412           0.450409
82          413           0.467698

[83 rows x 2 columns]
0.18352368902595287
-------------------------------------------------------------------------
    Test_Sample  Pred_Label(Price)
0           331           0.071884
1           332           0.390355
2           333           0.450409
3           334           0.241128
4           335           0.317561
..          ...                ...
78          409           0.112830
79          410           0.410373
80          411           0.432211
81          412           0.450409
82          413           0.467698

[83 rows x 2 columns]
0.04932542333779652


In [24]:
MAE(kNN_algorithm(train_data, train_labels, test_data, k = 3, distance_metric = 'manhattan'), test_labels)

print('-------------------------------------------------------------------------')

MSE(kNN_algorithm(train_data, train_labels, test_data, k = 3, distance_metric = 'manhattan'), test_labels)

    Test_Sample  Pred_Label(Price)
0           331           0.116166
1           332           0.340006
2           333           0.294207
3           334           0.301486
4           335           0.312405
..          ...                ...
78          409           0.102517
79          410           0.434334
80          411           0.353352
81          412           0.297847
82          413           0.445253

[83 rows x 2 columns]
0.20541213405920639
-------------------------------------------------------------------------
    Test_Sample  Pred_Label(Price)
0           331           0.116166
1           332           0.340006
2           333           0.294207
3           334           0.301486
4           335           0.312405
..          ...                ...
78          409           0.102517
79          410           0.434334
80          411           0.353352
81          412           0.297847
82          413           0.445253

[83 rows x 2 columns]
0.0658873764816906


In [25]:
MAE(kNN_algorithm(train_data, train_labels, test_data, k = 5, distance_metric = 'manhattan'), test_labels)

print('-------------------------------------------------------------------------')

MSE(kNN_algorithm(train_data, train_labels, test_data, k = 5, distance_metric = 'manhattan'), test_labels)

    Test_Sample  Pred_Label(Price)
0           331           0.238763
1           332           0.317743
2           333           0.245678
3           334           0.301729
4           335           0.278981
..          ...                ...
78          409           0.141037
79          410           0.432939
80          411           0.303549
81          412           0.315378
82          413           0.408189

[83 rows x 2 columns]
0.212154710627583
-------------------------------------------------------------------------
    Test_Sample  Pred_Label(Price)
0           331           0.238763
1           332           0.317743
2           333           0.245678
3           334           0.301729
4           335           0.278981
..          ...                ...
78          409           0.141037
79          410           0.432939
80          411           0.303549
81          412           0.315378
82          413           0.408189

[83 rows x 2 columns]
0.07016310980789908


In [26]:
MAE(kNN_algorithm(train_data, train_labels, test_data, k = 7, distance_metric = 'manhattan'), test_labels)

print('-------------------------------------------------------------------------')

MSE(kNN_algorithm(train_data, train_labels, test_data, k = 7, distance_metric = 'manhattan'), test_labels)

    Test_Sample  Pred_Label(Price)
0           331           0.254517
1           332           0.300143
2           333           0.270246
3           334           0.268686
4           335           0.280645
..          ...                ...
78          409           0.118289
79          410           0.389055
80          411           0.300923
81          412           0.276485
82          413           0.358638

[83 rows x 2 columns]
0.21376624430441157
-------------------------------------------------------------------------
    Test_Sample  Pred_Label(Price)
0           331           0.254517
1           332           0.300143
2           333           0.270246
3           334           0.268686
4           335           0.280645
..          ...                ...
78          409           0.118289
79          410           0.389055
80          411           0.300923
81          412           0.276485
82          413           0.358638

[83 rows x 2 columns]
0.07341020140868112




---



# Testing k = 1,3,5,7 with Euclidean distance metric:

In [27]:
MAE(kNN_algorithm(train_data, train_labels, test_data, k = 1, distance_metric = 'euclidean'), test_labels)

print('-------------------------------------------------------------------------')

MSE(kNN_algorithm(train_data, train_labels, test_data, k = 1, distance_metric = 'euclidean'), test_labels)

    Test_Sample  Pred_Label(Price)
0           331           0.071884
1           332           0.390355
2           333           0.450409
3           334           0.241128
4           335           0.317561
..          ...                ...
78          409           0.112830
79          410           0.337580
80          411           0.290264
81          412           0.450409
82          413           0.337580

[83 rows x 2 columns]
0.19967697954038216
-------------------------------------------------------------------------
    Test_Sample  Pred_Label(Price)
0           331           0.071884
1           332           0.390355
2           333           0.450409
3           334           0.241128
4           335           0.317561
..          ...                ...
78          409           0.112830
79          410           0.337580
80          411           0.290264
81          412           0.450409
82          413           0.337580

[83 rows x 2 columns]
0.05917145865540798


In [28]:
MAE(kNN_algorithm(train_data, train_labels, test_data, k = 3, distance_metric = 'euclidean'), test_labels)

print('-------------------------------------------------------------------------')

MSE(kNN_algorithm(train_data, train_labels, test_data, k = 3, distance_metric = 'euclidean'), test_labels)

    Test_Sample  Pred_Label(Price)
0           331           0.116166
1           332           0.239005
2           333           0.293904
3           334           0.352138
4           335           0.312405
..          ...                ...
78          409           0.102517
79          410           0.245981
80          411           0.359418
81          412           0.299363
82          413           0.383076

[83 rows x 2 columns]
0.21580165415634686
-------------------------------------------------------------------------
    Test_Sample  Pred_Label(Price)
0           331           0.116166
1           332           0.239005
2           333           0.293904
3           334           0.352138
4           335           0.312405
..          ...                ...
78          409           0.102517
79          410           0.245981
80          411           0.359418
81          412           0.299363
82          413           0.383076

[83 rows x 2 columns]
0.07249191013580412


In [29]:
MAE(kNN_algorithm(train_data, train_labels, test_data, k = 5, distance_metric = 'euclidean'), test_labels)

print('-------------------------------------------------------------------------')

MSE(kNN_algorithm(train_data, train_labels, test_data, k = 5, distance_metric = 'euclidean'), test_labels)

    Test_Sample  Pred_Label(Price)
0           331           0.238763
1           332           0.217834
2           333           0.295905
3           334           0.296451
4           335           0.278981
..          ...                ...
78          409           0.141037
79          410           0.219290
80          411           0.350864
81          412           0.252047
82          413           0.322657

[83 rows x 2 columns]
0.2131945153048675
-------------------------------------------------------------------------
    Test_Sample  Pred_Label(Price)
0           331           0.238763
1           332           0.217834
2           333           0.295905
3           334           0.296451
4           335           0.278981
..          ...                ...
78          409           0.141037
79          410           0.219290
80          411           0.350864
81          412           0.252047
82          413           0.322657

[83 rows x 2 columns]
0.07108003433966271


In [30]:
MAE(kNN_algorithm(train_data, train_labels, test_data, k = 7, distance_metric = 'euclidean'), test_labels)

print('-------------------------------------------------------------------------')

MSE(kNN_algorithm(train_data, train_labels, test_data, k = 7, distance_metric = 'euclidean'), test_labels)

    Test_Sample  Pred_Label(Price)
0           331           0.254517
1           332           0.245028
2           333           0.266736
3           334           0.319641
4           335           0.271806
..          ...                ...
78          409           0.147667
79          410           0.279735
80          411           0.315222
81          412           0.240088
82          413           0.331210

[83 rows x 2 columns]
0.21530519281377122
-------------------------------------------------------------------------
    Test_Sample  Pred_Label(Price)
0           331           0.254517
1           332           0.245028
2           333           0.266736
3           334           0.319641
4           335           0.271806
..          ...                ...
78          409           0.147667
79          410           0.279735
80          411           0.315222
81          412           0.240088
82          413           0.331210

[83 rows x 2 columns]
0.07338801738798188




---



Considering that lower values of MAE & MSE indicate the best performance, it would seem that k = 1 with the Manhattan distance metric is the best combination. 

# THAT'S ALL FOLKS



**Reminder: You can not use any library, with the exception of pandas and NumPy. So scikit-learn is forbidden!**

## Marking Scheme
**Program Correctness:** Your program should work correctly on all inputs (including new datasets). Also, if there are any specifications about how the program should be written, or how the output should appear, those specifications should be followed.

**Readability:** Variables functions should have meaningful names. The code should be organised into functions/methods where appropriate. There should be an appropriate amount of white space so that the code is readable, and the indentation should be consistent.

**Documentation:** your code and functions/methods should be appropriately commented. However, not every line should be commented because that makes your code overly busy. Think carefully about where comments are added.


**Code Elegance:** There are many ways to write the same functionality into your code, and some of them are needlessly slow or complicated. For example, if you are repeating the same code, it should be inside creating a new method/function or for loop

**Code efficiency:** The implementation is logically well-designed without inappropriate
design choices (e.g., unnecessary loops).